<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/02-Machine-Learning/notebooks/06-SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM

En esta notebook mostraremos el uso del clasificador **SVM** (Support Vector Machine). Realizaremos un ejemplo con datos artificiales, con fines didácticos, y un ejemplo más grande, con datos reales.

Usaremos la implementación de sklearn, llamada [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) (Support Vector Classifier)

## Ejemplo 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV

Funciones que necesitamos para graficar las fronteras de decisión

In [ ]:
#@title
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

### El conjunto de datos

Creamos un conjunto de datos con una condición XOR

In [ ]:
X = np.random.randn(1000, 2)
Y = np.array([int(np.logical_xor(x[0] > 0, x[1] > 0)) for x in X])

plt.figure()
plt.scatter(X[Y==0, 0], X[Y==0, 1], s=10, label='0')
plt.scatter(X[Y==1, 0], X[Y==1, 1], s=10, label='1')
plt.legend()
plt.xlabel('x1',fontsize=16)
plt.ylabel('x2',fontsize=16)
plt.show()

Separamos el conjunto de datos en train y test.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

print(f"X Train: {x_train.shape}")
print(f"X Test: {x_test.shape}")
print(f"Y Train: {y_train.shape}")
print(f"Y Test: {y_test.shape}")

X Train: (800, 2)
X Test: (200, 2)
Y Train: (800,)
Y Test: (200,)


### Clasificación

#### SVM lineal

In [ ]:
from sklearn.svm import SVC

lin_svm = SVC(kernel='linear')
lin_svm.fit(x_train, y_train)

# Performance
print(f"Training mean accuracy: {round(lin_svm.score(x_train, y_train),3)}")
print(f"Test mean accuracy: {round(lin_svm.score(x_test, y_test),3)}")

Training mean accuracy: 0.618
Test mean accuracy: 0.635


In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, ax = plt.subplots(dpi=100)  # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen
plot_contours(ax, lin_svm, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X[:,0], X[:,1], c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('x2', fontsize=16)
ax.set_xlabel('x1', fontsize=16)
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('Frontera de decisión del SVM')
plt.show()

#### ⭕ Probar otros kernels

Documentación: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
'''
1. Repetir el experimento de clasificación de arriba, usando otros kernels.
2. Graficar
3. ¿Qué kernel parece dar mejor resultado?
'''

'\n1. Repetir el experimento de clasificación de arriba, usando otros kernels.\n2. Graficar\n3. ¿Qué kernel parece dar mejor resultado?\n'

* El kernel lineal es mejor para datos linealmente separables. Es una opción cuando el conjunto de datos es grande. 
* El kernel Gaussiano (RBF) tiende a dar buenos resultados cuando no se tiene información adicional sobre los datos.
* Los kernels polinomiales tienden a dar buenos resultados cuando los datos de entrenamiento están normalizados.

### Usando gridsearch para encontrar los mejores parámetros

[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) toma un estimador (por ejemplo, SVM) y un conjunto de parámetros del estimador. Sobre estos parámetros hace una busqueda para encontrar la combinación de parámetros que da mejores resultados en el estimador. 

GridSearchCV tiene métodos “fit” y “score” method, entre otros. Es decir, no es necesario tomar los parámetros e introducirlos en el estimador.

Encuentra los mejores parámetros para el clasificador SVM utilizando grid search. Guíate por el desempeño en el set de entrenamiento y validación.

Prueba los siguientes hyperparámetros.
* kernel = linear, polynomial, rbf
* C = 0.01, 0.1, 1.0, 10, 100
* grado del polinomio = 1, 2, 3, 4 (solo para el kernel polinomial)
* gamma = auto, scale:

Definimos los parámetros sobre los que se hará la busqueda

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ('linear', 'poly', 'rbf'),
              'degree': [1, 2, 3, 4], 'gamma': ('auto', 'scale')}
param_grid

Realizamos una busqueda sobre estos parámetros 

In [ ]:
svc = SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(x_train, y_train)

In [ ]:
# Print info about best score and best hyperparameters
print(f"Best score: {clf.best_score_:.4f}")
print(f"Best params: {clf.best_params_}")

Best score: 0.9900
Best params: {'C': 100, 'degree': 2, 'gamma': 'auto', 'kernel': 'poly'}


In [ ]:
# Evaluate on the test set
best_svm = SVC(C=100, kernel='poly', degree=2, gamma='auto')
best_svm.fit(x_train, y_train)

print(f"Train mean accuracy: {best_svm.score(x_train, y_train):6.4f}")
print(f"Test mean accuracy: {best_svm.score(x_test, y_test):6.4f}")

Train mean accuracy: 0.9862
Test mean accuracy: 1.0000


Graficamos la frontera de decisión

In [ ]:
xx, yy = make_meshgrid(X[:,0], X[:,1]) # Hacemos el grid para graficar las regiones

fig, ax = plt.subplots(dpi=100)  # El parámetro dpi especifíca los puntos por pulgada (DPI) de la imagen
plot_contours(ax, best_svm, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X[:,0], X[:,1], c=Y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
ax.set_ylabel('x2', fontsize=16)
ax.set_xlabel('x1', fontsize=16)
ax.set_xticks(())
ax.set_yticks(())
ax.set_title('Frontera de decisión del SVM')
plt.show()

### Comparando el SVM lineal con el OLS (clasificador lineal)

En este ejercicio vamos a comparar la clasificación y la frontera de decisión del clasificador de la sesión anterior (discriminante lineal OLS) con el SVM con kernel lineal.

Para esto, vamos a usar ambos clasificadores en el mismo conjunto de datos. Después, compararemos la frontera de decisión.

Dado que el clasificador lo implementamos como una clase, podemos usarlo en esta notebook directamente. Hay dos maneras de hacerlo:

* Copiando el código y definiendo la clase otra vez:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

class LeastSquaresClassifier():
    def __init__(self, W:np.ndarray=None):
        '''
        W es la matriz de pesos, la cual puede ser especificada desde un principio, esto
        es opcional.
        '''
        self.W = W

    def encoderT(self, y:np.ndarray):
        K = np.max(y) + 1
        identidad = np.eye(K)
        return identidad[y] 

    def fit(self, X:np.ndarray, y:np.ndarray):
        '''
        Este método calcula la matriz de pesos para la matriz de puntos "aumentada" X
        y el conjunto de etiquetas y.
        '''
        T = self.encoderT(y)
        self.W = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ T
        
    def clasifica(self, X:np.ndarray):
        '''
        Este método predice las etiquetas para el conjunto de puntos X
        '''
        return np.argmax(X@self.W,axis=1)

* Importando la clase desde un archivo:

In [ ]:
# from clasificador_lineal import LeastSquaresClassifier

Definimos el conjunto de datos, usaremos un dataset de scikit-learn:

In [ ]:
from sklearn.datasets import make_moons

x_train, y_train = make_moons(n_samples = 120, random_state=89,noise=0.1)

#--- Lo graficamos para verlo ---
plt.figure()
plt.scatter(x_train[:,0], x_train[:,1], c=y_train)
plt.show()

⭕ Realiza la clasificación usando el clasificador OLS y grafica la frontera de decisión.

Puedes usar el código para clasificar y graficar que usamos en la sesión anterior

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

x1_test, x2_test = np.meshgrid(np.linspace(-2, 2.5, 100), np.linspace(-1, 1.5, 100))
x_test = np.array([x1_test, x2_test]).reshape(2, -1).T

features = PolynomialFeatures(1)
X_train = features.fit_transform(x_train)
X_test = features.fit_transform(x_test)

#------ COMPLETAR ------

#----------------------

plt.figure()
plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train)
plt.contour(x1_test, x2_test, y_ols.reshape(100, 100))
plt.tight_layout()
plt.show()

⭕ Ahora, usemos SVM lineal

Realiza la clasificación en el mismo dataset, usando SVM con kernel lineal y grafica la frontera de decisión. Puedes usar el código para clasificar y graficar que usamos anteriormente.

In [ ]:
x1_test, x2_test = np.meshgrid(np.linspace(-2, 2.5, 100), np.linspace(-1, 1.5, 100))
x_test = np.array([x1_test, x2_test]).reshape(2, -1).T

#------ COMPLETAR ------

#----------------------

plt.figure()
plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train)
plt.contour(x1_test, x2_test, y_svm.reshape(100, 100))
plt.tight_layout()
plt.show()


Dibujamos ambas FD juntas. 

In [ ]:
plt.figure()
#-----Dibujar los datos---------------------------------
plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train)
#-----Dibujar X_test (la malla de fondo para ver las regiones ------------
plt.contour(x1_test, x2_test, y_ols.reshape(100, 100),colors='green')
plt.contour(x1_test, x2_test, y_svm.reshape(100, 100),colors='blue')
plt.show()

Observar que no son la misma.

🔵 ¿Por qué no?

## Ejemplo 2

Para este problema usaremos el datset de Kaggle: [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)

**Contexto**

Los conjuntos de datos contienen transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos. Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está altamente desequilibrado, la clase positiva (fraudes) representa el 0.172% de todas las transacciones.

Contiene solo variables de entrada numéricas que son el resultado de una transformación PCA. Desafortunadamente, debido a problemas de confidencialidad, no se pueden obtener las características originales y más información de fondo sobre los datos. Las características $V_1$, $V_2$, ..., $V_{28}$ son los componentes principales obtenidos con PCA, las únicas características que no se han transformado con PCA son 'Tiempo' y 'Cantidad'. La función 'Tiempo' contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La característica 'Cantidad' es la Cantidad de la transacción, esta característica se puede utilizar para el aprendizaje sensible al costo dependiente del ejemplo. La característica 'Clase' es la variable de respuesta y toma el valor 1 en caso de fraude y 0 en caso contrario.

In [ ]:
!apt-get -qq install > /dev/null subversion

!svn checkout "https://github.com/DCDPUAEM/DCDP/trunk/02-Machine-Learning/data/"

Extraer el archivo zip

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

from zipfile import ZipFile 

archivo = "/content/data/creditcard.zip"

print('Extrayendo contenido...') 
with ZipFile(archivo, 'r') as Zip: 
    Zip.extractall() 
    print('Extracción finalizada.') 

credito = pd.read_csv("creditcard.csv")

In [ ]:
credito.head()

In [ ]:
credito.describe()

In [ ]:
plt.figure()

# Graficamos los que no son fraude
time_amount = credito[credito['Class'] == 0][['Time','Amount']].values
plt.scatter(time_amount[:,0], time_amount[:,1], 
            c='green',alpha=0.25,label='Clean')
# Graficamos los que sí son fraude
time_amount = credito[credito['Class'] == 1][['Time','Amount']].values
plt.scatter(time_amount[:,0], time_amount[:,1], 
            c='red',label='Fraud')
plt.legend(loc='best')
plt.title('Amount vs fraud')
plt.xlabel('Time', fontsize=16)
plt.ylabel('Amount', fontsize=16)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure()
sns.countplot(x = "Class", data = credito)
plt.show()

credito.loc[:, 'Class'].value_counts()

In [ ]:
No_of_frauds= len(credito[credito["Class"]==1])
No_of_normals = len(credito[credito["Class"]==0])
print("Hay {} transacciones fraudulentas ( Class 1)".format(No_of_frauds))
print("Hay {} transacciones normales ( Class 0)".format(No_of_normals))
total= No_of_frauds + No_of_normals
pf= (No_of_frauds / total)*100
pn= (No_of_normals / total)*100
print("Porcentaje clase 0 = {}%".format(np.round(pn,2)))
print("Porcentaje clase 1 = {}%".format(np.round(pf,2)))

### Submuestro

Se necesita hacer un submuestreo para balancear las clases

* Está claro que la Clase 1 está subrepresentada ya que solo  representa el 0.17% de todo el conjunto de datos. 
* Si entrenamos nuestro modelo usando este conjunto de datos, el modelo será ineficiente y será entrenado para predecir solo la Clase 0 porque no tendrá suficientes datos de entrenamiento.
* Podemos obtener una alta exactitud al probar el modelo, pero no debemos confundirnos con esto porque nuestro conjunto de datos no tiene datos de prueba equilibrados. Por lo tanto, tenemos que confiar en el recall que se basa en TP y FP.
* En los casos en que tengamos datos asimétricos, agregar datos adicionales de la característica subrepresentada (sobremuestreo) es una opción, mediante la modelación de la distribución de los datos. Por ahora no tenemos esa opción, así que tendremos que recurrir al submuestreo.
* El submuestreo del conjunto de datos implica mantener todos nuestros datos subrepresentados (Clase 1) mientras se muestrea el mismo número de características de la Clase 0 para crear un nuevo conjunto de datos que comprenda una representación igual de ambas clases.

Obtenemos un conjunto de datos más balanceado que contenga el doble de instancias no fraudulentas respecto a las fraudulentas

In [ ]:
#lista los indices de fraude
fraud_idxs = credito[credito["Class"]==1].index.to_list()

#lista de indices normales del data set completo
normal_idxs = credito[credito["Class"]==0].index.to_list()

#seleccion del numero de indices aleatorias igual al de transacciones fraudulentas
random_normal_idxs = np.random.choice(normal_idxs, No_of_frauds*2, replace= False)
random_normal_idxs = np.array(random_normal_idxs)

#concatena indices fraudulentos y normales para tener una lista de indices
undersampled_indices = np.concatenate([fraud_idxs, random_normal_idxs])

#usa la lista de indices sub-muestreados para obtener el data frame
undersampled_data = credito.iloc[undersampled_indices, :]

print(f"Fraude: {len(fraud_idxs)}, Normales: {len(random_normal_idxs)}")

undersampled_data.head()

Comprobemos que los datos quedaron balanceados

In [ ]:
No_of_frauds_sampled = len(undersampled_data[undersampled_data["Class"]== 1])

No_of_normals_sampled = len(undersampled_data[undersampled_data["Class"]== 0])

print("Número de transacciones fraudulentas (clase 1): ", No_of_frauds_sampled)
print("Número de transacciones normales (clase 0): ", No_of_normals_sampled)
total_sampled= No_of_frauds_sampled + No_of_normals_sampled
print("Número total de instancias: ", total_sampled)

Fraud_percent_sampled = (No_of_frauds_sampled / total_sampled)*100
Normal_percent_sampled = (No_of_normals_sampled / total_sampled)*100
print("Porcentaje clase 0 = ", Normal_percent_sampled)
print("Porcentaje clase 1 = ", Fraud_percent_sampled)


count_sampled = pd.value_counts(undersampled_data["Class"], sort= True)
count_sampled.plot(kind= 'bar')
plt.show()

### Oversampling

Ahora haremos un proceso llamado [SMOTE: Synthetic Minority Over-sampling Technique](https://arxiv.org/abs/1106.1813)


Para ello necesitamos instalar la librería de _aprendizaje desequilibrado_ ``imbalanced-learn`` de Python

In [ ]:
!pip install imbalanced-learn

In [ ]:
import imblearn
print(imblearn.__version__)

Re-escalemos los datos

In [ ]:
from sklearn import preprocessing

sc = preprocessing.StandardScaler()

undersampled_data.loc[:,"scaled_Amount"] = sc.fit_transform(undersampled_data["Amount"].values.reshape(-1,1))

# quitamos las columnas "Time" y "Amount"
undersampled_data.drop(["Time","Amount"], axis= 1,inplace=True)

undersampled_data.head()

⭕ Obtén la matriz de datos $X$ y el vector de clases $y$ correspondiente

Hagamos el proceso de sobre-muestreo [SMOTE](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

Verifiquemos la cantidad de datos ahora

In [ ]:
from collections import Counter

counter = Counter(y)
print(counter)

### Crear el conjunto de entrenamiento y prueba

Separa los datos en datos de entrenamiento (75%) y prueba (25%) 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 0)
print("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

⭕ Elige una SVM y entrénalo con un conjunto de parámetros de tu elección 

### Prueba el modelo 

Realiza las predicciones con el conjunto de prueba y bserva la matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)

CM = confusion_matrix(y_test, y_pred)

print(CM)

También podemos calcular las métricas de rendimiento *manualmente*.

In [ ]:
print("The accuracy is "+str((CM[1,1]+CM[0,0])/(CM[0,0] + CM[0,1]+CM[1,0] + CM[1,1])*100) + " %")
print("The recall from the confusion matrix is "+ str(CM[1,1]/(CM[1,0] + CM[1,1])*100) +" %")

⭕ Calcula también el *F1-score* y el *precision score*

### Aplica GridSearch para obtener los mejores parámetros para una SVM 

In [ ]:
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)

grid_search = grid_search.fit(X_train, y_train.ravel())
best_accuracy = grid_search.best_score_
print("The best accuracy using gridSearch is", best_accuracy)

best_parameters = grid_search.best_params_
print("The best parameters for using this model is", best_parameters)

### Utiliza los mejores parámetros para probar de nuevo tu modelo

In [ ]:
classifier_with_best_parameters =  SVC(C= best_parameters["C"], 
                                       kernel= best_parameters["kernel"], 
                                       random_state= 0)
classifier_with_best_parameters.fit(X_train, y_train.ravel())

#predicting the Class 
y_pred_best_parameters = classifier_with_best_parameters.predict(X_test)

CM2 = confusion_matrix(y_test, y_pred_best_parameters)
#visualizing the confusion matrix
print(CM2)

[[229  12]
 [ 23 228]]
The accuracy is 92.88617886178862 %
The recall from the confusion matrix is 90.83665338645417 %
[[237   4]
 [ 31 220]]
The accuracy is 92.88617886178862 %
The recall from the confusion matrix is 87.64940239043824 %


### ⭕ Prueba el modelo con el data set completo (sesgado)

Entrena un clasificador usando el dataset original sesgado, sin usar undersampling ni oversampling.

Puedes usar GridSearch (puede ser tardado). Al final, reportar las métricas de rendimiento y la matriz de confusión.

In [ ]:
#creating a new dataset to test our model
datanew = credito.copy()



___